In [1]:
import pandas as pd
from modeling_old import clear_column_names, regressor

In [2]:
def load_data(path="./df_2ap_final.csv"):
    df = pd.read_csv(path, header=[0, 1], index_col=None)

    y = df[("seq_time", "_")]
    X = df.drop(columns=[("seq_time", "_")])

    X.columns = clear_column_names(X)

    return X, y

In [3]:
X, y = load_data()
X_test, y_test = load_data("./df_1_2ap_test_final.csv")
X.shape, X_test.shape

((392, 212), (80, 212))

In [4]:
y_test

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
      ..
75   NaN
76   NaN
77   NaN
78   NaN
79   NaN
Name: (seq_time, _), Length: 80, dtype: float64

In [5]:
regressor(X, y, "xgboost")


Feature Importances:
                                        Feature  Importance
189         ap_from_ap_mean_rssi_le_nav_percent    0.471636
2    sta_to_ap_0_max_ant_rssi_in_nav_ed_percent    0.025487
196                   ap_from_ap_mean_rssi_mean    0.025147
129              sta_from_ap_1_max_ant_rssi_max    0.023095
74           sta_to_ap_1_mean_ant_rssi_skewness    0.021618
..                                          ...         ...
90             sta_from_ap_0_max_ant_rssi_range    0.000000
135              sta_from_ap_1_max_ant_rssi_var    0.000000
134             sta_from_ap_1_max_ant_rssi_mean    0.000000
132            sta_from_ap_1_max_ant_rssi_range    0.000000
148            sta_from_ap_1_mean_ant_rssi_sinr    0.000000

[212 rows x 2 columns]
{'mse': {'train': [2.0714014092346775, 2.396741464584394, 2.4237795368462876, 2.4879216858653663, 1.3588233725368501], 'test': [12.711487747706565, 9.44338761223745, 7.358799205542434, 14.435020926547203, 21.555509388748344]}, 'r2': {

{'shap_values':                                            Feature  SHAP Value  SHAP Sign
 189            ap_from_ap_mean_rssi_le_nav_percent    0.593302        1.0
 196                      ap_from_ap_mean_rssi_mean   -0.328717       -1.0
 168          ap_from_ap_max_rssi_in_nav_ed_percent   -0.156075       -1.0
 2       sta_to_ap_0_max_ant_rssi_in_nav_ed_percent    0.094768        1.0
 22   sta_to_ap_0_max_ant_rssi_wavelet_coefficients   -0.037450       -1.0
 ..                                             ...         ...        ...
 126   sta_from_ap_1_max_ant_rssi_in_nav_ed_percent    0.000000        0.0
 44          sta_to_ap_1_max_ant_rssi_la_ed_percent    0.000000        0.0
 111              sta_from_ap_0_mean_ant_rssi_range    0.000000        0.0
 27                sta_to_ap_0_mean_ant_rssi_median    0.000000        0.0
 211                                   protocol_udp    0.000000        0.0
 
 [212 rows x 3 columns],
 'feature_importances':                                   

In [6]:
import xgboost as xgb

model = xgb.XGBRegressor(
    objective="reg:squarederror",
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42,
)

model.fit(X, y)
y_test_pred = model.predict(X_test)

In [7]:
# save predictions
y_test_pred = pd.Series(y_test_pred, name="seq_time")
y_test_pred.to_csv("./results/y_1_2ap_test_pred.csv", index=False)